# HYSPLIT DATA PREPERATION

Greg Lee
<br>04.27.20

Summary: This notebook represents augmentations used on data in order to prepare it for HYSPLIT

**This is a rough approximation based upon quick google searches. No standardization performed (different websites used). 

** All other information comes from PUBCHEM

Slightly Soluble = 0.1<X<1.0 M
Insoluble = X<=0.1

In [1]:
#USE base env!

#Basic py: 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
import datetime

#Vis
#import seaborn as sns

#Geo
import geopandas as gpd
import fiona
from shapely.geometry import Point
import descartes
import contextily as ctx #Basemaps 

In [2]:
#load the data (please see TRI_Data_Exploration for more information)
tri_df = pd.read_csv('TRI_Carc_FSRel_9099.csv').drop(columns ='Unnamed: 0')

In [3]:
tri_df.head(2)

,CAS_No,Agent,Group,YEAR,TRIFD,FRSID,FACILITYNAME,CITY,COUNTY,ST,...,CAS#/COMPOUNDID,METAL,CARCINOGEN,UNITOFMEASURE,51-FUGITIVEAIR,52-STACKAIR,PRIMARYSIC,INDUSTRYSECTORCODE.1,88-ONE-TIMERELEASE,PRODUCTIONWSTE(81-87)
0,98828,Cumene,2B,1990,84103MCLCM474WE,110000469243,TESORO REFINING & MARKETING COMPANY LLC - SLC,SALT LAKE CITY,SALT LAKE,UT,...,98828,NO,YES,Pounds,0.0,64.0,2911,324,NaN,0.0
1,98828,Cumene,2B,1990,84066PNNZLWESTH,110000913218,PENNZOIL PRODUCTS CO ROOSEVELT REFINERY,ROOSEVELT,DUCHESNE,UT,...,98828,NO,YES,Pounds,100.0,100.0,2911,324,NaN,0.0


# Base Case: January 1990 

### Select the Data and Add Particle Information

In [4]:
#First let's load in the features to seperate the particles and gasses
TRI_chem_info = pd.read_csv('TRI_Carc_Half_Lives.csv')

#keep only those entries with complete information
TRI_chems = TRI_chem_info.dropna(subset = ['CAS'])
TRI_chems = TRI_chems.rename(columns = {'Unnamed: 8':'Phase'})

In [5]:
#Merge that data onto the tri dataframe
tri_df = pd.merge(tri_df,TRI_chem_info,left_on ='CAS_No',right_on = 'CAS',how='left')
tri_df = tri_df.drop(columns = ['CAS','Chems','88-ONE-TIMERELEASE'])

In [6]:
#Seperate into the known differential modeling variants:
fug_par_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['51-FUGITIVEAIR']>0) & (tri_df['Particle']==1)]
fug_gas_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['51-FUGITIVEAIR']>0) & (tri_df['Gas']==1) ]
stack_par_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['52-STACKAIR']>0) & (tri_df['Particle']==1)]
stack_gas_90 = tri_df[(tri_df['YEAR']==1990) & (tri_df['52-STACKAIR']>0) & (tri_df['Gas']==1) ]

print('Fugitive Particle Releases shape: ' + str(fug_par_90.shape))
print('Total number of unique chemicals: '+str(fug_par_90['CAS_No'].drop_duplicates().shape))
print('\nFugitive Gas Releases shape: ' + str(fug_gas_90.shape))
print('Total number of unique chemicals: '+str(fug_gas_90['CAS_No'].drop_duplicates().shape))
print('\nStack Particle Releases shape: ' + str(stack_par_90.shape))
print('Total number of unique chemicals: '+str(stack_par_90['CAS_No'].drop_duplicates().shape))
print('\nStack Gas Releases shape: ' + str(stack_gas_90.shape))
print('Total number of unique chemicals: '+str(stack_gas_90['CAS_No'].drop_duplicates().shape))

Fugitive Particle Releases shape: (13, 32)
Total number of unique chemicals: (3,)

Fugitive Gas Releases shape: (60, 32)
Total number of unique chemicals: (15,)

Stack Particle Releases shape: (12, 32)
Total number of unique chemicals: (5,)

Stack Gas Releases shape: (56, 32)
Total number of unique chemicals: (11,)


### Create a Calender of releases

Information for HYSPLIT
1. Lat/Lon
2. Convert to a datetime object (will need a value every three hours)

In [7]:
#Per each date, we will need to calculate 
cal = pd.date_range("1/1/90 00:00", "12/31/90 21:00", freq="3H")

#Check the shape of the frame
cal.shape[0] == 365*24/3

True

In [8]:
fug_par_90.columns
fug_par_90['UNITOFMEASURE'].drop_duplicates()

26    Pounds
Name: UNITOFMEASURE, dtype: object

In [9]:
final_df = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'])

for rows in range(fug_par_90.shape[0]):
    temp_df = pd.DataFrame({'Daily_Release': [fug_par_90['51-FUGITIVEAIR'].iloc[rows]/cal.shape[0]],
                            'Chem':fug_par_90['CHEMICAL'].iloc[rows],
                            'CAS':fug_par_90['CAS#/COMPOUNDID'].iloc[rows],
                            'MW':fug_par_90['MW'].iloc[rows],
                            'Half_Life':fug_par_90['1/2 Life'].iloc[rows],
                            'Solubility':fug_par_90['Solubility in H2O'].iloc[rows],
                            'Lat':fug_par_90['LATITUDE'].iloc[rows],
                            'Long':fug_par_90['LONGITUDE'].iloc[rows]})

    final_df = final_df.append(temp_df)

In [10]:
final = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'] )
for index in range(cal.shape[0]):
    final_df['Date_Time'] = cal[index].strftime("%m/%d/%Y %H:%M:%S")
    temp = final_df.set_index('Date_Time')
    final = final.append(temp)

In [11]:
idx = pd.MultiIndex.from_arrays([final.index.array,final['CAS'].array],names = ('Date','CAS'))
final = final.set_index(idx)
final = final.drop(columns = ['CAS'])

In [12]:
final.head()

Daily_Release         Chem      MW Half_Life  \
Date                CAS                                                   
01/01/1990 00:00:00 91203      11.491438  NAPHTHALENE  128.17   8 hours   
                    91203       3.424658  NAPHTHALENE  128.17   8 hours   
                    91203       0.127740  NAPHTHALENE  128.17   8 hours   
                    91203       0.018151  NAPHTHALENE  128.17   8 hours   
                    91203       0.001712  NAPHTHALENE  128.17   8 hours   

                           Solubility        Lat        Long  
Date                CAS                                       
01/01/1990 00:00:00 91203  Insoluble   40.297058 -111.734076  
                    91203  Insoluble   40.824900 -111.923800  
                    91203  Insoluble   40.788900 -111.904410  
                    91203  Insoluble   40.388555 -110.124910  
                    91203  Insoluble   40.886022 -111.904759

#### Let's build a function to do this: 

In [13]:
def chem_date_comb(startdate,enddate,freq,data):
    #Set the time dates for analysis: 
    calendar = pd.date_range(startdate,enddate, freq=freq)
    
    #Collect the daily information from the original data
    final_df = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'])
    for rows in range(data.shape[0]):
        
        #Edit if you want more information added to the release
        temp_df = pd.DataFrame({'Daily_Release': [data['51-FUGITIVEAIR'].iloc[rows]/cal.shape[0]],
                                'Chem':data['CHEMICAL'].iloc[rows],
                                'CAS':data['CAS#/COMPOUNDID'].iloc[rows],
                                'MW':data['MW'].iloc[rows],
                                'Half_Life':data['1/2 Life'].iloc[rows],
                                'Solubility':data['Solubility in H2O'].iloc[rows],
                                'Lat':data['LATITUDE'].iloc[rows],
                                'Long':data['LONGITUDE'].iloc[rows]})

        final_df = final_df.append(temp_df)
    
    #Add a date-time to each  
    final = pd.DataFrame(columns = ['Daily_Release','Chem','CAS','MW','Half_Life','Solubility','Lat','Long'] )
    for index in range(calendar.shape[0]):
        final_df['Date_Time'] = calendar[index].strftime("%m/%d/%Y %H:%M:%S")
        temp = final_df.set_index('Date_Time')
        final = final.append(temp)
    
    #Realign the axis:
    idx = pd.MultiIndex.from_arrays([final.index.array,final['CAS'].array],names = ('Date','CAS'))
    final = final.set_index(idx)
    
    return final.drop(columns = ['CAS'])

In [14]:
#Let's
fug_par_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = fug_par_90)

fug_gas_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = fug_gas_90)

stack_par_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = stack_par_90)

stack_gas_90_df = chem_date_comb(startdate = "1/1/90 00:00",
               enddate = "12/31/90 21:00",
               freq = "3H",
              data = stack_gas_90)

In [15]:
print(fug_par_90_df.shape)
print(fug_gas_90_df.shape)
print(stack_par_90_df.shape)
print(stack_gas_90_df.shape)

(37960, 7)
(175200, 7)
(35040, 7)
(163520, 7)


In [92]:
#Alright these shapes look good!
#pickle files for ease of access:
fug_par_90_df.to_pickle("./fug_particles_90_df.pkl")
fug_gas_90_df.to_pickle("./fug_gas_90_df.pkl")
stack_par_90_df.to_pickle("./stack_particles_90_df.pkl")
stack_gas_90_df.to_pickle("./stack_gas_90_df.pkl")

#Ready for use in HYSPLIT

## Modeling Fugitive Gas


In [17]:
test = fug_gas_90_df.loc['01/01/1990 00:00:00']

#So all the chemical releases should stay equal over the calendar year, so we should be able to reuse these inputs. 
#In order to output the right information for easy copy and paste will need the lat/long and height seperated by a space. 

new_col = []
location = []
for rows in range(test.shape[0]):
    line = test.iloc[rows]
    string=str(round(line['Lat'],2))+" "+str(round(line['Long'],1))+" "+"0.00"
    new_col.append(string)
    location.append('Location ' + str(rows+1))
test['Location'] = location
test['HS_Loc_set'] = new_col  

C:\Users\u0890227\Anaconda3\envs\lab\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\u0890227\Anaconda3\envs\lab\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
test

,Daily_Release,Chem,MW,Half_Life,Solubility,Lat,Long,Location,HS_Loc_set
CAS,,,,,,,,,
98828,0.034247,CUMENE,120.190,29 hrs,Insoluble,40.388555,-110.124910,Location 1,40.39 -110.1 0.00
91225,0.024658,QUINOLINE,129.160,21-160 days (temp dependent),Soluble (at high temps),40.297058,-111.734076,Location 2,40.3 -111.7 0.00
7664939,0.085616,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.707390,-111.892730,Location 3,40.71 -111.9 0.00
7664939,0.001712,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,41.241970,-111.978430,Location 4,41.24 -112.0 0.00
7664939,0.182192,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.587750,-111.969510,Location 5,40.59 -112.0 0.00
7664939,0.256849,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.344310,-111.742530,Location 6,40.34 -111.7 0.00
7664939,0.085616,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.824900,-111.923800,Location 7,40.82 -111.9 0.00
7664939,0.085616,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,41.260369,-112.231233,Location 8,41.26 -112.2 0.00
7664939,0.001712,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.725570,-112.000810,Location 9,40.73 -112.0 0.00


In [19]:
#Without batch file process, can only fit 30 chemicals within the input page

#It looks like the pollutants emitted from each source cannot be tailored. Thus per each chemical, a seperate file will
#have to be run. Per each chemical, a yearly calculation of the frequency should be possible, although this is currently not
#working. Throwing a stupido error. 

,Daily_Release,Chem,MW,Half_Life,Solubility,Lat,Long,Location,HS_Loc_set
CAS,,,,,,,,,
98828,0.034247,CUMENE,120.190,29 hrs,Insoluble,40.388555,-110.124910,Location 1,40.39 -110.1 0.00
91225,0.024658,QUINOLINE,129.160,21-160 days (temp dependent),Soluble (at high temps),40.297058,-111.734076,Location 2,40.3 -111.7 0.00
7664939,0.085616,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.707390,-111.892730,Location 3,40.71 -111.9 0.00
7664939,0.001712,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,41.241970,-111.978430,Location 4,41.24 -112.0 0.00
7664939,0.182192,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.587750,-111.969510,Location 5,40.59 -112.0 0.00
7664939,0.256849,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.344310,-111.742530,Location 6,40.34 -111.7 0.00
7664939,0.085616,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.824900,-111.923800,Location 7,40.82 -111.9 0.00
7664939,0.085616,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,41.260369,-112.231233,Location 8,41.26 -112.2 0.00
7664939,0.001712,"SULFURIC ACID (1994 AND AFTER ACID AEROSOLS"" O...",98.080,10-15 days,Soluble,40.725570,-112.000810,Location 9,40.73 -112.0 0.00


# Create batch file loop


In [45]:
import subprocess

In [69]:
#Test Creating a batch file within a loop


var = ['batchin','all','day']
for things in var:
myBat = open(r'C:\Users\u0890227\Desktop\test.bat','w+')
myBat.write('echo ' + var)
myBat.close() 

subprocess.call([r'C:\Users\u0890227\Desktop\test.bat'])
print('Script Completed')

0

In [31]:
myBat = open(r'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\batch\\hysplit_test.bat','w+')
myBat.write('''@echo off
setLocal EnableDelayedExpansion

REM -----------------------------------------------

REM Set the directories
set DIR=C:\\Users\\u0890227\\
set PGM=%DIR%\hysplit4
cd %PGM%\working

REM -----------------------------------------------

REM Checks for existence of ASCDATA file
IF EXIST ASCDATA.CFG DEL ASCDATA.CFG
echo -90.0   -180.0  lat/lon of lower left corner   >ASCDATA.CFG
echo 1.0     1.0     lat/lon spacing in degrees    >>ASCDATA.CFG
echo 180     360     lat/lon number of data points >>ASCDATA.CFG
echo 2               default land use category     >>ASCDATA.CFG
echo 0.2             default roughness length (m)  >>ASCDATA.CFG
echo '%PGM%\\bdyfiles\\'  directory of files         >>ASCDATA.CFG

REM -----------------------------------------------

REM Control file read by the model

REM Starting Time (year, month, day, hour)
echo 90 00 00 00             >CONTROL
REM Number of starting locations
echo 1                      >>CONTROL
REM Enter starting locations (lat, lon, meters)
echo 40.39 -110.1 0.00        >>CONTROL
REM Enter total run time (hours)
echo 48                     >>CONTROL
REM Vertical Motion (0:data 1:isob 3:dens 4:sigma 5:diverg 6:mls2agl 7:average 8: damped)
echo 0                      >>CONTROL
REM Top of the model
echo 5000.0                >>CONTROL
REM Number of input data grids
echo 1                      >>CONTROL
REM Meteorological data grid #1 directory
echo C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\       >>CONTROL
REM Meteorological data grid #1 file name
echo NARR199001            >>CONTROL

REM Number of different pollutants
echo 1                      >>CONTROL
REM Pollutant four character Identifier
echo CUMO                   >>CONTROL
REM Emission rate (per hour)
echo 0.034247               >>CONTROL
REM Hours of emission
echo 3.0                    >>CONTROL
REM Release start time: year month day hour minute --- 0s will be start time of the meteorological data file)
echo 00 00 00 00 00         >>CONTROL


REM Number of simultaneous concentration grids
echo 1                      >>CONTROL
REM Center (Lat, lon in degrees)
echo 0.0 0.0                >>CONTROL
REM Grid spacing (Lat,lon in degrees)
echo 0.05 0.05              >>CONTROL
REM Grid span (Lat,lon in degrees)
echo 30.0 30.0              >>CONTROL
REM Enter grid #1 directory (where things are written)
echo .\                     >>CONTROL
REM Enter grid #1 filename
echo cdump                  >>CONTROL
REM Number of vertical concentration levels (add 0 if adding treating at ground level)
echo 1                      >>CONTROL
REM Height of each level (m)
echo 100                    >>CONTROL
REM Sampling start time (year month day hour minute)
echo 00 00 00 00 00         >>CONTROL
REM Sampling stop time (year month day hour minute)
echo 00 00 00 00 00         >>CONTROL
REM Sampling interval (type hour minute) How often is a map created...
echo 00 12 00               >>CONTROL


REM Number of pullutants depositing
echo 1                      >>CONTROL
REM Particle: Diameter(um), Density (g,cc) and Shape
echo 0.0 0.0 0.0            >>CONTROL
REM Deposition Velocity (m/s), pollutant molecular weight (gram/mole), surface reactivity ratio, diffusivity ratio, effective henrys constant
echo 0.0 0.0 0.0 0.0 0.0    >>CONTROL
REM Wet Removal:Actual Henrys constant (In-cloud, below cloud)
echo 0.0 0.0 0.0            >>CONTROL
REM Radioactive decay half-life (days)
echo 0.0                    >>CONTROL
REM Pollutant Resuspension (1/m)
echo 0.0                    >>CONTROL

REM -----------------------------------------------
REM ADJUST SETTINGS (https://www.ready.noaa.gov/hysplitusersguide/S410.htm)

IF EXIST SETUP.CFG DEL SETUP.CFG
echo ^&SETUP                  >SETUP.CFG
REM Particle vs Puff (currently only particle)
echo initd = 0                >>SETUP.CFG
REM Total number of particles
echo numpar = 1000           >>SETUP.CFG
echo /                       >>SETUP.CFG


REM -----------------------------------------------
IF EXIST cdump DEL cdump
REM -----------------------------------------------

REM Run the model
%PGM%\exec\hycs_std

REM -----------------------------------------------

REM execute the plotting
ECHO 'TITLE^&','### %0 ### ^&' >LABELS.CFG

REM adding -a2 to imply Arcview generate in value
REM adding -x1.0E+12 to make the units picopounds
REM Will need to adjust values..I think..
%PGM%\exec\concplot -icdump -a2 -c50 -x1.0E+12 -j%PGM%\graphics\arlmap

REM Open the plotting
concplot.ps

REM -----------------------------------------------
REM saving to a shape file: (THIS IS WORKING)
REM Units are too small to be detected

IF EXIST GIS_00100_ps_01.txt (
     IF EXIST concplot.shp DEL concplot.???
     %PGM%\exec\ascii2shp -d concplot polygons <GIS_00100_ps_01.txt
     %PGM%\exec\txt2dbf -C11 -C5 -C9 -C5 -C6../ -C6 -d,  GIS_00100_ps_01.att concplot.dbf

)'''.format(start_model = '90 00 00 00', 
            num_loc = '1', 
            start_locs = '40.39 -110.1 0.00', 
            tot_runtime = '48', 
            mod_top = '5000.0'))
myBat.close()

#subprocess.call([r'C:\Users\u0890227\Desktop\test.bat'])
#print('Script Completed')

In [43]:
#Version 2
myBat = open(r'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\batch\\hysplit_test.bat','w+')
myBat.write('''@echo off
setLocal EnableDelayedExpansion

REM -----------------------------------------------

REM Set the directories
set DIR=C:\\Users\\u0890227\\
set PGM=%DIR%\hysplit4
cd %PGM%\working

REM -----------------------------------------------

REM Checks for existence of ASCDATA file
IF EXIST ASCDATA.CFG DEL ASCDATA.CFG
echo -90.0   -180.0  lat/lon of lower left corner   >ASCDATA.CFG
echo 1.0     1.0     lat/lon spacing in degrees    >>ASCDATA.CFG
echo 180     360     lat/lon number of data points >>ASCDATA.CFG
echo 2               default land use category     >>ASCDATA.CFG
echo 0.2             default roughness length (m)  >>ASCDATA.CFG
echo '%PGM%\\bdyfiles\\'  directory of files         >>ASCDATA.CFG

REM -----------------------------------------------

REM Control file read by the model
            
REM Starting Time (year, month, day, hour)
echo {start_model}             >CONTROL
REM Number of starting locations
echo {num_loc}                      >>CONTROL
REM Enter starting locations (lat, lon, meters)
echo {start_locs}        >>CONTROL
REM Enter total run time (hours)
echo {tot_runtime}                     >>CONTROL
REM Vertical Motion (0:data 1:isob 3:dens 4:sigma 5:diverg 6:mls2agl 7:average 8: damped)
echo 0                      >>CONTROL
REM Top of the model
echo {mod_top}                >>CONTROL
REM Number of input data grids
echo {input_data_grids}                      >>CONTROL
REM Meteorological data grid #1 directory and file name
echo {grid_1_dir}       >>CONTROL
echo {grid_1_name}            >>CONTROL
REM Meteorological data grid #1 directory and file name
echo {grid_2_dir}       >>CONTROL
echo {grid_2_name}            >>CONTROL

REM Number of different pollutants
echo 1                      >>CONTROL
REM Pollutant four character Identifier
echo {chem_ident}                   >>CONTROL
REM Emission rate (per hour)
echo {emis_rate}               >>CONTROL
REM Hours of emission
echo {emis_hours}                    >>CONTROL
REM Release start time: year month day hour minute --- 0s will be start time of the meteorological data file)
echo 00 00 00 00 00         >>CONTROL


REM Number of simultaneous concentration grids
echo 1                      >>CONTROL
REM Center (Lat, lon in degrees)
echo 0.0 0.0                >>CONTROL
REM Grid spacing (Lat,lon in degrees)
echo 0.05 0.05              >>CONTROL
REM Grid span (Lat,lon in degrees)
echo 30.0 30.0              >>CONTROL
REM Enter grid #1 directory (where things are written)
echo .\                     >>CONTROL
REM Enter grid #1 filename
echo cdump                  >>CONTROL
REM Number of vertical concentration levels (add 0 if adding treating at ground level)
echo 1                      >>CONTROL
REM Height of each level (m)
echo 100                    >>CONTROL
REM Sampling start time (year month day hour minute)
echo 00 00 00 00 00         >>CONTROL
REM Sampling stop time (year month day hour minute)
echo 00 00 00 00 00         >>CONTROL
REM Sampling interval (type hour minute) How often is a map created...
echo {samp_interval}               >>CONTROL


REM Number of pullutants depositing
echo 1                      >>CONTROL
REM Particle: Diameter(um), Density (g,cc) and Shape
echo 0.0 0.0 0.0            >>CONTROL
REM Deposition Velocity (m/s), pollutant molecular weight (gram/mole), surface reactivity ratio, diffusivity ratio, effective henrys constant
echo 0.0 0.0 0.0 0.0 0.0    >>CONTROL
REM Wet Removal:Actual Henrys constant (In-cloud, below cloud)
echo 0.0 0.0 0.0            >>CONTROL
REM Radioactive decay half-life (days)
echo 0.0                    >>CONTROL
REM Pollutant Resuspension (1/m)
echo 0.0                    >>CONTROL

REM -----------------------------------------------
REM ADJUST SETTINGS (https://www.ready.noaa.gov/hysplitusersguide/S410.htm)

IF EXIST SETUP.CFG DEL SETUP.CFG
echo ^&SETUP                  >SETUP.CFG
REM Particle vs Puff (currently only particle)
echo initd = {puff_or_part}                >>SETUP.CFG
REM Total number of particles
echo numpar = {particle_num}           >>SETUP.CFG
echo /                       >>SETUP.CFG


REM -----------------------------------------------
IF EXIST cdump DEL cdump
REM -----------------------------------------------

REM Run the model
%PGM%\exec\hycs_std

REM -----------------------------------------------

REM execute the plotting
ECHO 'TITLE^&','### %0 ### ^&' >LABELS.CFG

REM adding -a2 to imply Arcview generate in value
REM adding -x1.0E+12 to make the units picopounds
REM Will need to adjust values..I think..
%PGM%\exec\concplot -icdump -a2 -c50 -x1.0E+12 -j%PGM%\graphics\arlmap

REM Open the plotting
concplot.ps

REM -----------------------------------------------
REM saving to a shape file: (THIS IS WORKING)
REM Units are too small to be detected

IF EXIST GIS_00100_ps_01.txt (
     IF EXIST concplot.shp DEL concplot.???
     %PGM%\exec\ascii2shp -d concplot polygons <GIS_00100_ps_01.txt
     %PGM%\exec\txt2dbf -C11 -C5 -C9 -C5 -C6../ -C6 -d,  GIS_00100_ps_01.att concplot.dbf

)'''.format(start_model = '90 00 00 00', 
            num_loc = '1', 
            start_locs = '40.39 -110.1 0.00', 
            tot_runtime = '48', 
            mod_top = '5000.0',
            input_data_grids = '2',
            grid_1_dir = "C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\",
            grid_1_name = 'NARR199001',
            grid_2_dir = 'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\',
            grid_2_name = 'NARR199002',
            chem_ident = 'CUMO',
            emis_rate = '0.03427',
            emis_hours = '3.0000',
            samp_interval = '00 03 00',
            puff_or_part = '0',
            particle_num = '1000'))
myBat.close()

#subprocess.call([r'C:\Users\u0890227\Desktop\test.bat'])
#print('Script Completed')

In [49]:
myBat = open(r'C:\Users\u0890227\Desktop\test.bat','w+')
myBat.write('''
                @echo off
                setLocal EnableDelayedExpansion
                echo Hello World
                
                for %%f in (*.in) do (
                echo f
                )
                ''')
myBat.close() 

subprocess.call([r'C:\Users\u0890227\Desktop\test.bat'])
print('Script Completed')

Script Completed


In [11]:
#Version 3
myBat = open(r'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\batch\\hysplit_test_v2.bat','w+')
myBat.write('''@echo off
setLocal EnableDelayedExpansion

REM -----------------------------------------------

REM Set the directories
set DIR=C:\\Users\\u0890227\\
set PGM=%DIR%\hysplit4
cd %PGM%\working

REM -----------------------------------------------

REM Checks for existence of ASCDATA file
IF EXIST ASCDATA.CFG DEL ASCDATA.CFG
echo -90.0   -180.0  lat/lon of lower left corner   >ASCDATA.CFG
echo 1.0     1.0     lat/lon spacing in degrees    >>ASCDATA.CFG
echo 180     360     lat/lon number of data points >>ASCDATA.CFG
echo 2               default land use category     >>ASCDATA.CFG
echo 0.2             default roughness length (m)  >>ASCDATA.CFG
echo '%PGM%\\bdyfiles\\'  directory of files         >>ASCDATA.CFG

REM -----------------------------------------------

REM Control file read by the model
            
REM Starting Time (year, month, day, hour)
echo {start_model}             >CONTROL
REM Number of starting locations
echo {num_loc}                      >>CONTROL
REM Enter starting locations (lat, lon, meters)
echo {start_locs}        >>CONTROL
REM Enter total run time (hours)
echo {tot_runtime}                     >>CONTROL
REM Vertical Motion (0:data 1:isob 3:dens 4:sigma 5:diverg 6:mls2agl 7:average 8: damped)
echo 0                      >>CONTROL
REM Top of the model
echo {mod_top}                >>CONTROL
REM Number of input data grids
echo {input_data_grids}                      >>CONTROL
REM Meteorological data grid #1 directory and file name
echo {grid_1_dir}       >>CONTROL
echo {grid_1_name}            >>CONTROL
REM Meteorological data grid #1 directory and file name
echo {grid_2_dir}       >>CONTROL
echo {grid_2_name}            >>CONTROL

REM Number of different pollutants
echo 1                      >>CONTROL
REM Pollutant four character Identifier
echo {chem_ident}                   >>CONTROL
REM Emission rate (per hour)
echo {emis_rate}               >>CONTROL
REM Hours of emission
echo {emis_hours}                    >>CONTROL
REM Release start time: year month day hour minute --- 0s will be start time of the meteorological data file)
echo 00 00 00 00 00         >>CONTROL


REM Number of simultaneous concentration grids
echo 1                      >>CONTROL
REM Center (Lat, lon in degrees)
echo 0.0 0.0                >>CONTROL
REM Grid spacing (Lat,lon in degrees)
echo 0.05 0.05              >>CONTROL
REM Grid span (Lat,lon in degrees)
echo 30.0 30.0              >>CONTROL
REM Enter grid #1 directory (where things are written)
echo .\                     >>CONTROL
REM Enter grid #1 filename
echo cdump                  >>CONTROL
REM Number of vertical concentration levels (add 0 if adding treating at ground level)
echo 1                      >>CONTROL
REM Height of each level (m)
echo 100                    >>CONTROL
REM Sampling start time (year month day hour minute)
echo 00 00 00 00 00         >>CONTROL
REM Sampling stop time (year month day hour minute)
echo 00 00 00 00 00         >>CONTROL
REM Sampling interval (type hour minute) How often is a map created...
echo {samp_interval}               >>CONTROL


REM Number of pullutants depositing
echo 1                      >>CONTROL
REM Particle: Diameter(um), Density (g,cc) and Shape
echo 0.0 0.0 0.0            >>CONTROL
REM Deposition Velocity (m/s), pollutant molecular weight (gram/mole), surface reactivity ratio, diffusivity ratio, effective henrys constant
echo 0.0 0.0 0.0 0.0 0.0    >>CONTROL
REM Wet Removal:Actual Henrys constant (In-cloud, below cloud)
echo 0.0 0.0 0.0            >>CONTROL
REM Radioactive decay half-life (days)
echo 0.0                    >>CONTROL
REM Pollutant Resuspension (1/m)
echo 0.0                    >>CONTROL

REM -----------------------------------------------
REM ADJUST SETTINGS (https://www.ready.noaa.gov/hysplitusersguide/S410.htm)

IF EXIST SETUP.CFG DEL SETUP.CFG
echo ^&SETUP                  >SETUP.CFG
REM Particle vs Puff (currently only particle)
echo initd = {puff_or_part}                >>SETUP.CFG
REM Total number of particles
echo numpar = {particle_num}           >>SETUP.CFG
echo /                       >>SETUP.CFG


REM -----------------------------------------------
IF EXIST cdump DEL cdump
REM -----------------------------------------------

REM Run the model
%PGM%\\exec\\hycs_std

REM -----------------------------------------------

REM execute the plotting
ECHO 'TITLE^&','### %0 ### ^&' >LABELS.CFG

REM adding -a2 to imply Arcview generate in value
REM adding -x1.0E+12 to make the units picopounds
REM Will need to adjust values..I think..
%PGM%\exec\concplot -icdump -a2 -c50 -x1.0E+12 -j%PGM%\graphics\arlmap

REM Open the plotting


REM --------------------------------------------------
REM Save all shapefiles into shapefiles
set /A Counter=0
set /A file_count = 0

REM this allows me to print all the GIS files used to create shapefiles
for %%i in (C:\\Users\\u0890227\\hysplit4\\working\\*GIS_*) do (
  set /A Counter+=1

  if !Counter! ==1 (
  set att=%%i
  )

  if !Counter! ==2 (
  set txt=%%i
  set /A file_count+=1

    echo !att!
    echo !txt!

    %PGM%\\exec\\ascii2shp -d concpolys polygons <!txt!
    %PGM%\\exec\\txt2dbf -C11 -C5 -C9 -C5 -C6../ -C6 -d, !att! concpolys.dbf

    mkdir C:\\Users\\u0890227\\Desktop\\hysplit\\{filename}\\test_!file_count!
    move %PGM%\\working\\*concpolys* C:\\Users\\u0890227\\Desktop\\hysplit\\{filename}\\test_!file_count!

    set /A Counter=0
  )
)

concplot.ps

'''.format(start_model = '90 00 00 00', 
            num_loc = '1', 
            start_locs = '40.39 -110.1 0.00', 
            tot_runtime = '48', 
            mod_top = '5000.0',
            input_data_grids = '2',
            grid_1_dir = "C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\",
            grid_1_name = 'NARR199001',
            grid_2_dir = 'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\',
            grid_2_name = 'NARR199002',
            chem_ident = 'CUMO',
            emis_rate = '0.03427',
            emis_hours = '3.0000',
            samp_interval = '00 03 00',
            puff_or_part = '0',
            particle_num = '1000',
            filename = 'Jan_1'
           ))
myBat.close()

#subprocess.call([r'C:\\Users\\u0890227\\Desktop\\test.bat'])
#print('Script Completed')

In [90]:
def HYSPLIT_configure(start_model,
                      num_loc,
                      start_locs,
                      tot_runtime,
                      mod_top,
                      input_data_grids,
                      grid_1_dir,
                      grid_1_name,
                      grid_2_dir,
                      grid_2_name,
                      chem_ident,
                      emis_rate,
                      emis_hours,
                      samp_interval,
                      puff_or_part,
                      particle_num,
                      filename):
    
    import subprocess
    #Create the Batch File 
    myBat = open(r'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\batch\\hysplit_temp.bat','w+')
    
    #Write the Control file
    myBat.write('''@echo off
    setLocal EnableDelayedExpansion

    REM -----------------------------------------------

    REM Set the directories
    set DIR=C:\\Users\\u0890227\\
    set PGM=%DIR%\hysplit4
    cd %PGM%\working

    REM -----------------------------------------------

    REM Checks for existence of ASCDATA file
    IF EXIST ASCDATA.CFG DEL ASCDATA.CFG
    echo -90.0   -180.0  lat/lon of lower left corner   >ASCDATA.CFG
    echo 1.0     1.0     lat/lon spacing in degrees    >>ASCDATA.CFG
    echo 180     360     lat/lon number of data points >>ASCDATA.CFG
    echo 2               default land use category     >>ASCDATA.CFG
    echo 0.2             default roughness length (m)  >>ASCDATA.CFG
    echo '%PGM%\\bdyfiles\\'  directory of files         >>ASCDATA.CFG

    REM -----------------------------------------------

    REM Control file read by the model

    REM Starting Time (year, month, day, hour)
    echo {start_model}             >CONTROL
    REM Number of starting locations
    echo {num_loc}                      >>CONTROL
    REM Enter starting locations (lat, lon, meters)
    echo {start_locs}        >>CONTROL
    REM Enter total run time (hours)
    echo {tot_runtime}                     >>CONTROL
    REM Vertical Motion (0:data 1:isob 3:dens 4:sigma 5:diverg 6:mls2agl 7:average 8: damped)
    echo 0                      >>CONTROL
    REM Top of the model
    echo {mod_top}                >>CONTROL
    REM Number of input data grids
    echo {input_data_grids}                      >>CONTROL
    REM Meteorological data grid #1 directory and file name
    echo {grid_1_dir}       >>CONTROL
    echo {grid_1_name}            >>CONTROL
    REM Meteorological data grid #1 directory and file name
    echo {grid_2_dir}       >>CONTROL
    echo {grid_2_name}            >>CONTROL

    REM Number of different pollutants
    echo 1                      >>CONTROL
    REM Pollutant four character Identifier
    echo {chem_ident}                   >>CONTROL
    REM Emission rate (per hour)
    echo {emis_rate}               >>CONTROL
    REM Hours of emission
    echo {emis_hours}                    >>CONTROL
    REM Release start time: year month day hour minute --- 0s will be start time of the meteorological data file)
    echo 00 00 00 00 00         >>CONTROL


    REM Number of simultaneous concentration grids
    echo 1                      >>CONTROL
    REM Center (Lat, lon in degrees)
    echo 0.0 0.0                >>CONTROL
    REM Grid spacing (Lat,lon in degrees)
    echo 0.05 0.05              >>CONTROL
    REM Grid span (Lat,lon in degrees)
    echo 30.0 30.0              >>CONTROL
    REM Enter grid #1 directory (where things are written)
    echo .\                     >>CONTROL
    REM Enter grid #1 filename
    echo cdump                  >>CONTROL
    REM Number of vertical concentration levels (add 0 if adding treating at ground level)
    echo 1                      >>CONTROL
    REM Height of each level (m)
    echo 100                    >>CONTROL
    REM Sampling start time (year month day hour minute)
    echo 00 00 00 00 00         >>CONTROL
    REM Sampling stop time (year month day hour minute)
    echo 00 00 00 00 00         >>CONTROL
    REM Sampling interval (type hour minute) How often is a map created...
    echo {samp_interval}               >>CONTROL


    REM Number of pullutants depositing
    echo 1                      >>CONTROL
    REM Particle: Diameter(um), Density (g,cc) and Shape
    echo 0.0 0.0 0.0            >>CONTROL
    REM Deposition Velocity (m/s), pollutant molecular weight (gram/mole), surface reactivity ratio, diffusivity ratio, effective henrys constant
    echo 0.0 0.0 0.0 0.0 0.0    >>CONTROL
    REM Wet Removal:Actual Henrys constant (In-cloud, below cloud)
    echo 0.0 0.0 0.0            >>CONTROL
    REM Radioactive decay half-life (days)
    echo 0.0                    >>CONTROL
    REM Pollutant Resuspension (1/m)
    echo 0.0                    >>CONTROL

    REM -----------------------------------------------
    REM ADJUST SETTINGS (https://www.ready.noaa.gov/hysplitusersguide/S410.htm)

    IF EXIST SETUP.CFG DEL SETUP.CFG
    echo ^&SETUP                  >SETUP.CFG
    REM Particle vs Puff (currently only particle)
    echo initd = {puff_or_part}                >>SETUP.CFG
    REM Total number of particles
    echo numpar = {particle_num}           >>SETUP.CFG
    echo /                       >>SETUP.CFG


    REM -----------------------------------------------
    IF EXIST cdump DEL cdump
    REM -----------------------------------------------

    REM Run the model
    %PGM%\\exec\\hycs_std

    REM -----------------------------------------------

    REM execute the plotting
    ECHO 'TITLE^&','### %0 ### ^&' >LABELS.CFG

    REM adding -a2 to imply Arcview generate in value
    REM adding -x1.0E+12 to make the units picopounds
    REM Will need to adjust values..I think..
    %PGM%\exec\concplot -icdump -a2 -c50 -x1.0E+12 -j%PGM%\graphics\arlmap

    REM Open the plotting


    REM --------------------------------------------------
    REM Save all shapefiles into shapefiles
    set /A Counter=0
    set /A file_count = 0

    REM this allows me to print all the GIS files used to create shapefiles
    for %%i in (C:\\Users\\u0890227\\hysplit4\\working\\*GIS_*) do (
      set /A Counter+=1

      if !Counter! ==1 (
      set att=%%i
      )

      if !Counter! ==2 (
      set txt=%%i
      set /A file_count+=3

        echo !att!
        echo !txt!

        %PGM%\\exec\\ascii2shp -d concpolys polygons <!txt!
        %PGM%\\exec\\txt2dbf -C11 -C5 -C9 -C5 -C6../ -C6 -d, !att! concpolys.dbf
        
        
        mkdir C:\\Users\\u0890227\\Desktop\\hysplit\\{filename}\\hour_after_start_!file_count!
        move %PGM%\\working\\*concpolys* C:\\Users\\u0890227\\Desktop\\hysplit\\{filename}\\hour_after_start_!file_count!

        set /A Counter=0
      )
    )

    REM concplot.ps

    '''.format(start_model = start_model,
               num_loc = num_loc,
               start_locs = start_locs,
               tot_runtime = tot_runtime,
               mod_top = mod_top,
               input_data_grids = input_data_grids,
               grid_1_dir=grid_1_dir,
               grid_1_name = grid_1_name,
               grid_2_dir = grid_2_dir,
               grid_2_name = grid_2_name,
               chem_ident = chem_ident,
               emis_rate = emis_rate,
               emis_hours = emis_hours,
               samp_interval = samp_interval,
               puff_or_part = puff_or_part,
               particle_num = particle_num,
               filename = filename
              ))

    myBat.close()

    subprocess.call([r'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\batch\\hysplit_temp.bat'])
    print('Script Completed') 

In [91]:
HYSPLIT_configure(start_model = '90 00 00 00', 
                    num_loc = '1', 
                    start_locs = '40.39 -110.1 0.00', 
                    tot_runtime = '48', 
                    mod_top = '5000.0',
                    input_data_grids = '2',
                    grid_1_dir = "C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\",
                    grid_1_name = 'NARR199001',
                    grid_2_dir = 'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\',
                    grid_2_name = 'NARR199002',
                    chem_ident = 'CUMO',
                    emis_rate = '0.03427',
                    emis_hours = '3.0000',
                    samp_interval = '00 03 00',
                    puff_or_part = '0',
                    particle_num = '1000',
                    filename = 'Jan_1')

Script Completed


In [39]:
fug_gas_90_df

Daily_Release  \
Date                CAS                      
01/01/1990 00:00:00 98828         0.034247   
                    91225         0.024658   
                    7664939       0.085616   
                    7664939       0.001712   
                    7664939       0.182192   
...                                    ...   
12/31/1990 21:00:00 100414        1.449658   
                    100414        0.078767   
                    100414        0.426027   
                    100414        0.001712   
                    100414        0.537329   

                                                                          Chem  \
Date                CAS                                                          
01/01/1990 00:00:00 98828                                               CUMENE   
                    91225                                            QUINOLINE   
                    7664939  SULFURIC ACID (1994 AND AFTER ACID AEROSOLS" O...   
                    7664939  SULFURIC ACID (1994 AND AFTER ACID AEROSOLS" O...   
                    7664939  SULFURIC ACID (1994 AND AFTER ACID AEROSOLS" O...   
...                                                                        ...   
12/31/1990 21:00:00 100414                                        ETHYLBENZENE   
                    100414                                        ETHYLBENZENE   
                    100414                                        ETHYLBENZENE   
                    100414                                        ETHYLBENZENE   
                    100414                                        ETHYLBENZENE   

                                 MW                     Half_Life  \
Date                CAS                                             
01/01/1990 00:00:00 98828    120.19                        29 hrs   
                    91225    129.16  21-160 days (temp dependent)   
                    7664939   98.08                    10-15 days   
                    7664939   98.08                    10-15 days   
                    7664939   98.08                    10-15 days   
...                             ...                           ...   
12/31/1990 21:00:00 100414   106.16                      24 hours   
                    100414   106.16                      24 hours   
                    100414   106.16                      24 hours   
                    100414   106.16                      24 hours   
                    100414   106.16                      24 hours   

                                          Solubility        Lat        Long  
Date                CAS                                                      
01/01/1990 00:00:00 98828                  Insoluble  40.388555 -110.124910  
                    91225    Soluble (at high temps)  40.297058 -111.734076  
                    7664939                  Soluble  40.707390 -111.892730  
                    7664939                  Soluble  41.241970 -111.978430  
                    7664939                  Soluble  40.587750 -111.969510  
...                                              ...        ...         ...  
12/31/1990 21:00:00 100414                 Insoluble  40.788900 -111.904410  
                    100414                 Insoluble  40.388555 -110.124910  
                    100414                 Insoluble  40.886022 -111.904759  
                    100414                 Insoluble  40.866310 -111.911870  
                    100414                 Insoluble  40.838386 -111.921443  

[175200 rows x 7 columns]

## Loop for Date HYSPLIT Configuration 

I am setting this up to be based off the date, in case we need to edit the dates at which there is release. Additionally, I've pickled the TRI dataframes for computational ease:

In [93]:
fug_par_90_df= pd.read_pickle("./fug_particles_90_df.pkl")
fug_gas_90_df= pd.read_pickle("./fug_gas_90_df.pkl")
stack_par_90_df = pd.read_pickle("./stack_particles_90_df.pkl")
stack_gas_90_df = pd.read_pickle("./stack_gas_90_df.pkl")

In [ ]:
cumene_test = fug_gas_90_df[fug_gas_90_df['Chem']=='CUMENE']

In [40]:
lat_long_format = []

for rows in range(cumene_test.shape[0]):
    line = cumene_test.iloc[rows]
    #This is fugitive releases as it sets the height of the release at 0
    string=str(round(line['Lat'],2))+" "+str(round(line['Long'],1))+" "+"0.00"
    lat_long_format.append(string)
cumene_test['HS_Loc_set'] = lat_long_format  

C:\Users\u0890227\Anaconda3\envs\lab\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
cumene_samp = cumene_test.reset_index()[0:248]
counter = 0 

for rows in range(cumene_samp.shape[0]):
    counter +=1
    release = str(cumene_samp.iloc[rows]['Daily_Release'])
    Chem = str(cumene_samp.iloc[rows]['Chem'][:4])
    lat_lon_height = str(cumene_samp.iloc[rows]['HS_Loc_set'])
    date = str(cumene_samp.iloc[rows]['Date'])
    year = date[8:10]
    month = date[0:2]
    day = date[3:5]
    hour = date[11:13]
    date = year+' '+month + ' '+day+' '+hour
    save_name = Chem+'_'+year+'_'+month+'_'+day+'_'+hour
    
    HYSPLIT_configure(start_model = date, 
                    num_loc = '1', 
                    start_locs = lat_lon_height, 
                    tot_runtime = '48', 
                    mod_top = '5000.0',
                    input_data_grids = '2',
                    grid_1_dir = "C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\",
                    grid_1_name = 'NARR199001',
                    grid_2_dir = 'C:\\Users\\u0890227\\Desktop\\HYSPLIT\\NARR_Weather_Data\\',
                    grid_2_name = 'NARR199002',
                    chem_ident = Chem,
                    emis_rate = release,
                    emis_hours = '3.0000',
                    samp_interval = '00 03 00',
                    puff_or_part = '0',
                    particle_num = '1000',
                    filename = save_name)
    
    if counter ==3:
        break
    

Script Completed
Script Completed
Script Completed


In [87]:
!dir C:\Users\u0890227\Desktop\HYSPLIT\

 Volume in drive C is Windows
 Volume Serial Number is 48B0-2796

 Directory of C:\Users\u0890227\Desktop\HYSPLIT

05/11/2020  03:31 PM    <DIR>          .
05/11/2020  03:31 PM    <DIR>          ..
05/11/2020  01:36 PM    <DIR>          batch
05/11/2020  03:30 PM    <DIR>          CUME_90_01_01_00
05/11/2020  03:30 PM    <DIR>          CUME_90_01_01_03
05/11/2020  03:31 PM    <DIR>          CUME_90_01_01_06
05/11/2020  01:44 PM    <DIR>          Jan_1
04/22/2020  03:32 PM    <DIR>          NARR_Weather_Data
05/11/2020  10:02 AM    <DIR>          shapefiles
               0 File(s)              0 bytes
               9 Dir(s)  275,380,760,576 bytes free
